# Getting COVID-19 vaccination numbers to August 2021

In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib import rc
from datetime import datetime
from datetime import date
from pandas.tseries.offsets import DateOffset
import math as mth
import datetime as dt

# Understanding and format vaccination numbers


In [2]:
# this reads the filee what we are getting from the Israel Ministry of Health
# the file was drawn from https://data.gov.il/dataset/covid-19 in August
vaccination = pd.read_csv('D:/MAXCORlabwork/covid/most_current_csvs/vaccinated-per-day-2021-08-24.csv')

In [4]:
vaccination['date_as_date'] = pd.to_datetime(vaccination['VaccinationDate'], format= '%Y-%m-%d')
vaccination.head(5)

,VaccinationDate,age_group,first_dose,second_dose,third_dose,date_as_date
0,2020-12-20,20-29,392,0,0,2020-12-20
1,2020-12-20,0-19,21,0,0,2020-12-20
2,2020-12-20,80-89,94,0,0,2020-12-20
3,2020-12-20,60-69,1516,0,0,2020-12-20
4,2020-12-20,50-59,1787,0,0,2020-12-20


In [6]:
# the data was provided as strings. we willconvert the strings to number
# and we will conservatively claim that when the number is recorded as '<15' it can
# be rounded down to zero. we do not want to overestimate the vaccinated population
# we will return to this assumption later and run the numbers if we round up to compare
vaccination['second_dose'] = vaccination['second_dose'].str.replace('<15', '0')
vaccination['first_dose'] = vaccination['first_dose'].str.replace('<15', '0')
vaccination['first_dose'] = pd.to_numeric(vaccination['first_dose'])
vaccination['second_dose'] = pd.to_numeric(vaccination['second_dose'])

### At present file the MOH put in first and second doses as strings and third as a numeric.
Watch out for changes and change code accordingly if the code is to be reporposed for later datasets

In [5]:
age_grouped_to19_vax = vaccination[vaccination.age_group == '0-19'].copy(deep = True)
age_grouped_to29_vax = vaccination[vaccination.age_group == '20-29'].copy(deep = True)
age_grouped_to39_vax = vaccination[vaccination.age_group == '30-39'].copy(deep = True)
age_grouped_to49_vax = vaccination[vaccination.age_group == '40-49'].copy(deep = True)
age_grouped_to59_vax = vaccination[vaccination.age_group == '50-59'].copy(deep = True)
age_grouped_to69_vax = vaccination[vaccination.age_group == '60-69'].copy(deep = True)
age_grouped_to79_vax = vaccination[vaccination.age_group == '70-79'].copy(deep = True)
age_grouped_to89_vax = vaccination[vaccination.age_group == '80-89'].copy(deep = True)
age_grouped_90plus_vax = vaccination[vaccination.age_group == '90+'].copy(deep = True)

In [6]:
age_grouped_to19_vax = age_grouped_to19_vax.rename(
    columns = {'first_dose':'first_dose_in_0to19',
                'second_dose': 'second_dose_in_0to19',
                'third_dose':'third_dose_in_0to19'}
    )
age_grouped_to29_vax = age_grouped_to29_vax.rename(
    columns = {'first_dose':'first_dose_in_20to29',
                'second_dose': 'second_dose_in_20to29',
                'third_dose':'third_dose_in_20to29'}
    )
age_grouped_to39_vax = age_grouped_to39_vax.rename(
    columns = {'first_dose':'first_dose_in_30to39',
                'second_dose': 'second_dose_in_30to39',
                'third_dose':'third_dose_in_30to39'}
    )
age_grouped_to49_vax = age_grouped_to49_vax.rename(
    columns = {'first_dose':'first_dose_in_40to49',
                'second_dose': 'second_dose_in_40to49',
                'third_dose':'third_dose_in_40to49'}
    )
age_grouped_to59_vax = age_grouped_to59_vax.rename(
    columns = {'first_dose':'first_dose_in_50to59',
                'second_dose': 'second_dose_in_50to59',
                'third_dose':'third_dose_in_50to59'}
    )
age_grouped_to69_vax = age_grouped_to69_vax.rename(
    columns = {'first_dose':'first_dose_in_60to69',
                'second_dose': 'second_dose_in_60to69',
                'third_dose':'third_dose_in_60to69'}
    )
age_grouped_to79_vax = age_grouped_to79_vax.rename(
    columns = {'first_dose':'first_dose_in_70to79',
                'second_dose': 'second_dose_in_70to79',
                'third_dose':'third_dose_in_70to79'}
    )
age_grouped_to89_vax = age_grouped_to89_vax.rename(
    columns = {'first_dose':'first_dose_in_80to89',
                'second_dose': 'second_dose_in_80to89',
                'third_dose':'third_dose_in_80to89'}
    )
age_grouped_90plus_vax = age_grouped_90plus_vax.rename(
    columns = {'first_dose':'first_dose_in_90plus',
                'second_dose': 'second_dose_in_90plus',
                'third_dose':'third_dose_in_90plus'}
    )



In [7]:
age_grouped_to19_vax = age_grouped_to19_vax.drop('age_group', axis = 1)
age_grouped_to29_vax = age_grouped_to29_vax.drop('age_group', axis = 1)
age_grouped_to39_vax = age_grouped_to39_vax.drop('age_group', axis = 1)
age_grouped_to49_vax = age_grouped_to49_vax.drop('age_group', axis = 1)
age_grouped_to59_vax = age_grouped_to59_vax.drop('age_group', axis = 1)
age_grouped_to69_vax = age_grouped_to69_vax.drop('age_group', axis = 1)
age_grouped_to79_vax = age_grouped_to79_vax.drop('age_group', axis = 1)
age_grouped_to89_vax = age_grouped_to89_vax.drop('age_group', axis = 1)
age_grouped_90plus_vax= age_grouped_90plus_vax.drop('age_group', axis = 1)

In [8]:
# now we can look at first or second doses by age group
# let's run an example
age_grouped_90plus_vax

,VaccinationDate,first_dose_in_90plus,second_dose_in_90plus,third_dose_in_90plus,date_as_date
6,2020-12-20,<15,0,0,2020-12-20
16,2020-12-21,167,0,0,2020-12-21
21,2020-12-22,456,0,0,2020-12-22
30,2020-12-23,1026,0,0,2020-12-23
40,2020-12-24,1349,0,0,2020-12-24
...,...,...,...,...,...
2175,2021-08-20,<15,<15,295,2021-08-20
2184,2021-08-21,<15,<15,137,2021-08-21
2192,2021-08-22,23,15,575,2021-08-22
2199,2021-08-23,27,<15,568,2021-08-23


In [9]:
vax_by_age = age_grouped_to19_vax.merge(age_grouped_to29_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to39_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to49_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to59_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to69_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to79_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_to89_vax, on = 'date_as_date')
vax_by_age = vax_by_age.merge(age_grouped_90plus_vax, on = 'date_as_date')

D:\bin\anaconda3\envs\jupyter_environment\lib\site-packages\pandas\core\frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'VaccinationDate_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [10]:
vax_by_age

,VaccinationDate_x,first_dose_in_0to19,second_dose_in_0to19,third_dose_in_0to19,date_as_date,VaccinationDate_y,first_dose_in_20to29,second_dose_in_20to29,third_dose_in_20to29,VaccinationDate_x,...,second_dose_in_70to79,third_dose_in_70to79,VaccinationDate_y,first_dose_in_80to89,second_dose_in_80to89,third_dose_in_80to89,VaccinationDate,first_dose_in_90plus,second_dose_in_90plus,third_dose_in_90plus
0,2020-12-20,21,0,0,2020-12-20,2020-12-20,392,0,0,2020-12-20,...,0,0,2020-12-20,94,0,0,2020-12-20,<15,0,0
1,2020-12-21,67,0,0,2020-12-21,2020-12-21,1064,0,0,2020-12-21,...,0,0,2020-12-21,1164,0,0,2020-12-21,167,0,0
2,2020-12-22,100,0,0,2020-12-22,2020-12-22,1575,0,0,2020-12-22,...,0,0,2020-12-22,3498,0,0,2020-12-22,456,0,0
3,2020-12-23,160,0,0,2020-12-23,2020-12-23,2121,0,0,2020-12-23,...,0,0,2020-12-23,5677,0,0,2020-12-23,1026,0,0
4,2020-12-24,202,0,0,2020-12-24,2020-12-24,2069,0,0,2020-12-24,...,0,0,2020-12-24,7678,0,0,2020-12-24,1349,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,2021-08-20,3220,1377,0,2021-08-20,2021-08-20,353,253,0,2021-08-20,...,58,2938,2021-08-20,32,16,1407,2021-08-20,<15,<15,295
231,2021-08-21,2618,717,0,2021-08-21,2021-08-21,240,153,0,2021-08-21,...,30,1432,2021-08-21,<15,<15,666,2021-08-21,<15,<15,137
232,2021-08-22,6365,2679,0,2021-08-22,2021-08-22,918,365,0,2021-08-22,...,130,5928,2021-08-22,92,56,2700,2021-08-22,23,15,575
233,2021-08-23,7178,2255,0,2021-08-23,2021-08-23,946,356,0,2021-08-23,...,136,4983,2021-08-23,72,58,2429,2021-08-23,27,<15,568


In [11]:
vax_by_age.columns

Index(['VaccinationDate_x', 'first_dose_in_0to19', 'second_dose_in_0to19',
       'third_dose_in_0to19', 'date_as_date', 'VaccinationDate_y',
       'first_dose_in_20to29', 'second_dose_in_20to29', 'third_dose_in_20to29',
       'VaccinationDate_x', 'first_dose_in_30to39', 'second_dose_in_30to39',
       'third_dose_in_30to39', 'VaccinationDate_y', 'first_dose_in_40to49',
       'second_dose_in_40to49', 'third_dose_in_40to49', 'VaccinationDate_x',
       'first_dose_in_50to59', 'second_dose_in_50to59', 'third_dose_in_50to59',
       'VaccinationDate_y', 'first_dose_in_60to69', 'second_dose_in_60to69',
       'third_dose_in_60to69', 'VaccinationDate_x', 'first_dose_in_70to79',
       'second_dose_in_70to79', 'third_dose_in_70to79', 'VaccinationDate_y',
       'first_dose_in_80to89', 'second_dose_in_80to89', 'third_dose_in_80to89',
       'VaccinationDate', 'first_dose_in_90plus', 'second_dose_in_90plus',
       'third_dose_in_90plus'],
      dtype='object')

In [12]:
first_dose_by_age = vax_by_age[['date_as_date',
                                'first_dose_in_0to19',
                                'first_dose_in_20to29',
                                'first_dose_in_30to39', 
                                'first_dose_in_40to49',
                                'first_dose_in_50to59',
                                'first_dose_in_60to69',
                                'first_dose_in_70to79',
                                'first_dose_in_80to89',
                                'first_dose_in_90plus',]]

In [13]:
first_dose_by_age

,date_as_date,first_dose_in_0to19,first_dose_in_20to29,first_dose_in_30to39,first_dose_in_40to49,first_dose_in_50to59,first_dose_in_60to69,first_dose_in_70to79,first_dose_in_80to89,first_dose_in_90plus
0,2020-12-20,21,392,1196,1867,1787,1516,494,94,<15
1,2020-12-21,67,1064,2779,3786,3690,7282,4887,1164,167
2,2020-12-22,100,1575,3591,4531,4663,14433,11763,3498,456
3,2020-12-23,160,2121,3847,5038,5336,21612,18008,5677,1026
4,2020-12-24,202,2069,3785,5196,5660,25639,21761,7678,1349
...,...,...,...,...,...,...,...,...,...,...
230,2021-08-20,3220,353,347,319,260,132,75,32,<15
231,2021-08-21,2618,240,177,232,136,50,30,<15,<15
232,2021-08-22,6365,918,769,799,499,295,152,92,23
233,2021-08-23,7178,946,889,907,530,278,156,72,27


In [14]:
first_dose_by_age.to_csv('first_dose_by_age.csv')

In [15]:
second_dose_by_age = vax_by_age[[
    'date_as_date',
    'second_dose_in_0to19', 
    'second_dose_in_20to29',
    'second_dose_in_30to39',
    'second_dose_in_40to49',
    'second_dose_in_50to59', 
    'second_dose_in_60to69',
    'second_dose_in_70to79',
    'second_dose_in_80to89',
    'second_dose_in_90plus',]]

In [16]:
second_dose_by_age#.to_csv('first_dose_by_age.csv')

,date_as_date,second_dose_in_0to19,second_dose_in_20to29,second_dose_in_30to39,second_dose_in_40to49,second_dose_in_50to59,second_dose_in_60to69,second_dose_in_70to79,second_dose_in_80to89,second_dose_in_90plus
0,2020-12-20,0,0,0,0,0,0,0,0,0
1,2020-12-21,0,0,0,0,0,0,0,0,0
2,2020-12-22,0,0,0,0,0,0,0,0,0
3,2020-12-23,0,0,0,0,0,0,0,0,0
4,2020-12-24,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
230,2021-08-20,1377,253,229,296,333,109,58,16,<15
231,2021-08-21,717,153,114,218,163,82,30,<15,<15
232,2021-08-22,2679,365,330,562,394,220,130,56,15
233,2021-08-23,2255,356,323,534,388,226,136,58,<15


In [17]:
second_dose_by_age.to_csv('second_dose_by_age.csv')

In [18]:
third_dose_by_age = vax_by_age[[
    'date_as_date',
    'third_dose_in_0to19', 
    'third_dose_in_20to29',
    'third_dose_in_30to39',
    'third_dose_in_40to49',
    'third_dose_in_50to59', 
    'third_dose_in_60to69',
    'third_dose_in_70to79',
    'third_dose_in_80to89',
    'third_dose_in_90plus',]]

In [19]:
third_dose_by_age

,date_as_date,third_dose_in_0to19,third_dose_in_20to29,third_dose_in_30to39,third_dose_in_40to49,third_dose_in_50to59,third_dose_in_60to69,third_dose_in_70to79,third_dose_in_80to89,third_dose_in_90plus
0,2020-12-20,0,0,0,0,0,0,0,0,0
1,2020-12-21,0,0,0,0,0,0,0,0,0
2,2020-12-22,0,0,0,0,0,0,0,0,0
3,2020-12-23,0,0,0,0,0,0,0,0,0
4,2020-12-24,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
230,2021-08-20,0,0,1275,26206,21029,6673,2938,1407,295
231,2021-08-21,0,0,1442,31502,11924,3112,1432,666,137
232,2021-08-22,0,0,5756,51181,25855,10519,5928,2700,575
233,2021-08-23,0,0,8630,43410,21243,8932,4983,2429,568


In [20]:
third_dose_by_age.to_csv('third_dose_by_age.csv')

### Now you have csv of first second and third doses. We can also make a big spreadsheet of everything from vax_by_age

In [21]:
vax_by_age.to_csv('big_vax_spreadsheet.csv')